In [1]:
# 📦 Instalar dependencias necesarias
!pip install pyarrow --quiet

# 🚗 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📚 Librerías necesarias
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm

# 📍 Rutas de entrada y salida
ruta_espacioF = "/content/drive/MyDrive/Datos/9_1EspacioF.parquet"
ruta_parametros = "/content/drive/MyDrive/Datos/9_2ParametrosFuncional.pkl"
ruta_matriz = "/content/drive/MyDrive/Datos/10_Matriz_Distancias_Funcional.npy"
ruta_nits = "/content/drive/MyDrive/Datos/10_Orden_NITs_Funcional.npy"

# 📥 Cargar base funcional (espacio ℱ)
espacioF = pd.read_parquet(ruta_espacioF)
print(f"✅ Espacio funcional cargado: {espacioF.shape}")

# 📥 Cargar parámetros óptimos
with open(ruta_parametros, "rb") as f:
    params = pickle.load(f)

k = params["k"]
lambda_p = params["lambda"]
pesos = params["pesos"]

# 🎯 Columnas funcionales y variables
columnas_funcionales = [col for col in espacioF.columns if "_-" in col]
indicadores = sorted(set(col.split("_")[0] for col in columnas_funcionales))
n_ventana = len(set(col.split("_")[1] for col in columnas_funcionales))

# 🧠 Definir función de distancia funcional con pesos
def distancia_ponderada(f1, f2, lambda_p, n, pesos):
    total, suma_pesos = 0, 0
    for var in indicadores:
        v1 = [f1.get(f"{var}_-{i}", np.nan) for i in range(n)]
        v2 = [f2.get(f"{var}_-{i}", np.nan) for i in range(n)]
        l1, validos = 0, 0
        for a, b in zip(v1, v2):
            if pd.notna(a) and pd.notna(b):
                if np.isinf(a) and np.isinf(b) and a == b:
                    l1 += 0
                elif np.isinf(a) or np.isinf(b):
                    l1 += np.inf
                else:
                    l1 += abs(a - b)
                validos += 1
        faltantes = n - validos
        if validos > 0:
            penalizada = l1 * (1 + lambda_p * (faltantes / n))
            acotada = penalizada / (1 + penalizada) if np.isfinite(penalizada) else 1.0
        else:
            acotada = 1.0
        total += pesos[var] * acotada
        suma_pesos += pesos[var]
    return total / suma_pesos if suma_pesos > 0 else 1.0

# 🔢 Crear matriz vacía y calcular distancias
n_empresas = espacioF.shape[0]
orden_nits = espacioF.index.tolist()
matriz = np.zeros((n_empresas, n_empresas))

print("🔧 Calculando matriz de distancias funcionales...")
for i in tqdm(range(n_empresas), desc="↔️ Comparando empresas"):
    f1 = espacioF.iloc[i]
    for j in range(i + 1, n_empresas):
        f2 = espacioF.iloc[j]
        d = distancia_ponderada(f1, f2, lambda_p, n_ventana, pesos)
        matriz[i, j] = d
        matriz[j, i] = d  # simetría

# 💾 Guardar matriz y orden de NITs
np.save(ruta_matriz, matriz)
np.save(ruta_nits, np.array(orden_nits))

print(f"\n✅ Matriz de distancias guardada en: {ruta_matriz}")
print(f"✅ Orden de NITs guardado en: {ruta_nits}")
# 📝 Crear resumen final del Paso 10
ruta_resumen = "/content/drive/MyDrive/Datos/10_Resumen_Distancias.txt"

with open(ruta_resumen, "w") as f:
    f.write("📄 Resumen de cálculo de la matriz de distancias funcionales\n")
    f.write("=================================================================\n\n")

    f.write("✅ Archivos utilizados:\n")
    f.write(f"   - Base funcional: {ruta_espacioF.split('/')[-1]}\n")
    f.write(f"   - Parámetros óptimos: {ruta_parametros.split('/')[-1]}\n\n")

    f.write("📐 Parámetros de la métrica:\n")
    f.write(f"   - k: {k}\n")
    f.write(f"   - lambda: {lambda_p:.4f}\n")
    f.write(f"   - Número de indicadores: {len(indicadores)}\n")
    f.write(f"   - Tamaño de ventana: {n_ventana} años\n\n")

    f.write("💾 Archivos generados:\n")
    f.write(f"   - Matriz de distancias: {ruta_matriz.split('/')[-1]}\n")
    f.write(f"   - Orden de NITs: {ruta_nits.split('/')[-1]}\n\n")

    f.write(f"📊 Número total de empresas: {n_empresas:,}\n")
    f.write(f"📊 Dimensión de la matriz: {matriz.shape}\n")

print(f"📝 Resumen guardado en: {ruta_resumen}")


Mounted at /content/drive
✅ Espacio funcional cargado: (5565, 89)
🔧 Calculando matriz de distancias funcionales...


↔️ Comparando empresas:   0%|          | 0/5565 [00:00<?, ?it/s]


✅ Matriz de distancias guardada en: /content/drive/MyDrive/Datos/10_Matriz_Distancias_Funcional.npy
✅ Orden de NITs guardado en: /content/drive/MyDrive/Datos/10_Orden_NITs_Funcional.npy
📝 Resumen guardado en: /content/drive/MyDrive/Datos/10_Resumen_Distancias.txt
